# Transformers

Autoregressive large language models are build on top of transformers architecture.

Transformer is a neural network architecture based on self-attention mechanisms, avoiding thus recurrence 
connections. This way sequence problems can be solved in parallel, making training large models faster.

Self attention mechanisms relies on compute similarity between embeddings. For NLP they can create relations between words helps a given task such as translation.

I think self attention mechanisms works as learning to read without knowing grammar. For example, "The boy kicked the ball and it disappered." Attention layer can process "it" while at same time
is looking for other words. With time, I believe the layer will start to perceive that when objects appears in a sentece "it" will be appearing to replace or determine the object itself. This is what a pronoum does. If this happens without be parallel, it will be very dificult to perceive this relations.

To understand in detail the attention mechanisms The Attention is All You Need paper [2] is a good place to
understand the transformer architecture.

I create my personal finds and placed on this repositoy https://github.com/ramonlins/obsidian/tree/master/Papers/transformers/Ashish%20Vaswani.

To access the material is necessary to install obisidian and excalidraw plugin.

In attempt to create a simple example to implement transforms from scratch I created a simple 
overview of the architecture abstracting some parts such as residual connections, stack of layers and multiple heads.

![transformer](./images/transformer.png "Figure 1")

The idea here is to create a squence model to predict the next character vowel based on a sequence of three characters extracted from the portugues alphabet. 


In this simpler ilustration the encoder is composed by a attention layer connected with a feed forward network; 
Its outputs are connected with another attention layer togeter with the output of the masked attention layer; This attention layer output is connected to another feedforward layer that pass through a linear and
softmax operation.

The input sequence is first embedded into word vectors using an embedding layer. These word vectors serve as inputs for both query and key transformations.

Separate linear transformations with learned weight matrices are then applied to these word embeddings to obtain query vectors (Q) and key vectors (K). The purpose of these separate transformations is to project each word's embedding into lower-dimensional spaces that capture different aspects of their meaning.

SQ and K embeddings differ because they undergo distinct linear transformations with unique weight matrices during computation.

In the Transformer model, during training, the Q (query) matrix learns to search for relevant information based on what K (keys) are showing as reference.

The similarities are highlighted passing trough softmax working like amplifiers to the synaptic connections for a given input and task in matrix V (the data itself).


## Finding next vowel

pt-alphabet is defined by:

"abcdefghijklmnopqrstuvwxyz"

In [1]:
import torch

Dataset

In [2]:
characters = "-abcdefghijklmnopqrstuvwxyz#"
emb = {}
pos = {}
i2c = {}
for i, c in enumerate(characters):
    pos[c] = i         # position i mapping
    i2c[i] = c
    #emb[c] = format(i,'05b') # binary encoding mapping        

In [3]:
pos

{'-': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '#': 27}

In [4]:
i2c

{0: '-',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 27: '#'}

In [5]:
def generate_combinations(characters, length):
    def generate_combinations_recursive(current_combination, remaining_characters):
        if len(current_combination) == length:
            combinations.append(current_combination)
            return
        for i in range(len(remaining_characters)):
            generate_combinations_recursive(
                current_combination + remaining_characters[i],
                remaining_characters[i + 1:]
            )

    combinations = []
    generate_combinations_recursive('', characters)
    return combinations

length = 3

combinations = generate_combinations(characters, length)

for combo in combinations:
    print(combo)


-ab
-ac
-ad
-ae
-af
-ag
-ah
-ai
-aj
-ak
-al
-am
-an
-ao
-ap
-aq
-ar
-as
-at
-au
-av
-aw
-ax
-ay
-az
-a#
-bc
-bd
-be
-bf
-bg
-bh
-bi
-bj
-bk
-bl
-bm
-bn
-bo
-bp
-bq
-br
-bs
-bt
-bu
-bv
-bw
-bx
-by
-bz
-b#
-cd
-ce
-cf
-cg
-ch
-ci
-cj
-ck
-cl
-cm
-cn
-co
-cp
-cq
-cr
-cs
-ct
-cu
-cv
-cw
-cx
-cy
-cz
-c#
-de
-df
-dg
-dh
-di
-dj
-dk
-dl
-dm
-dn
-do
-dp
-dq
-dr
-ds
-dt
-du
-dv
-dw
-dx
-dy
-dz
-d#
-ef
-eg
-eh
-ei
-ej
-ek
-el
-em
-en
-eo
-ep
-eq
-er
-es
-et
-eu
-ev
-ew
-ex
-ey
-ez
-e#
-fg
-fh
-fi
-fj
-fk
-fl
-fm
-fn
-fo
-fp
-fq
-fr
-fs
-ft
-fu
-fv
-fw
-fx
-fy
-fz
-f#
-gh
-gi
-gj
-gk
-gl
-gm
-gn
-go
-gp
-gq
-gr
-gs
-gt
-gu
-gv
-gw
-gx
-gy
-gz
-g#
-hi
-hj
-hk
-hl
-hm
-hn
-ho
-hp
-hq
-hr
-hs
-ht
-hu
-hv
-hw
-hx
-hy
-hz
-h#
-ij
-ik
-il
-im
-in
-io
-ip
-iq
-ir
-is
-it
-iu
-iv
-iw
-ix
-iy
-iz
-i#
-jk
-jl
-jm
-jn
-jo
-jp
-jq
-jr
-js
-jt
-ju
-jv
-jw
-jx
-jy
-jz
-j#
-kl
-km
-kn
-ko
-kp
-kq
-kr
-ks
-kt
-ku
-kv
-kw
-kx
-ky
-kz
-k#
-lm
-ln
-lo
-lp
-lq
-lr
-ls
-lt
-lu
-lv
-lw
-lx
-ly
-lz
-l#
-mn
-mo
-mp
-mq


In [6]:
len(characters)

28

In [7]:
vowels = 'aeiou#'

n_seq = 3
n = len(characters)

v2i = {}
for i, v in enumerate(characters):
    if v in vowels:
        v2i[v] = i

In [8]:
n

28

In [9]:
v2i

{'a': 1, 'e': 5, 'i': 9, 'o': 15, 'u': 21, '#': 27}

In [10]:
i2v = {}

for i, v in enumerate(vowels):
   i2v[i] = v
i2v

{0: 'a', 1: 'e', 2: 'i', 3: 'o', 4: 'u', 5: '#'}

In [11]:
# Create all possible combinations of input and ouputs
valid_seqs = []       
samples = []
valid_seqs = []
num_of_samples = len(combinations)
for combination in combinations:
    character_idx = pos[combination[-1]]
    for vowel, vowel_idx in v2i.items():
        if vowel_idx > character_idx:
            input_seq = []
            for character in combination:
                input_seq.append(pos[character])
            
            valid_seqs.append(input_seq)    
            input_seq_t = torch.tensor(input_seq)
            next_vowel_t = torch.tensor([vowel_idx])
    
            samples.append([input_seq_t, next_vowel_t])
            
            break
        
        if character_idx > pos['u']:
            input_seq = []
            for character in combination:
                input_seq.append(pos[character])
            
            valid_seqs.append(input_seq)
            input_seq_t = torch.tensor(input_seq)
            next_vowel_t = torch.tensor([pos['#']])
    
            samples.append([input_seq_t, next_vowel_t])
            break

In [12]:
pos['u']

21

In [13]:
samples

[[tensor([0, 1, 2]), tensor([5])],
 [tensor([0, 1, 3]), tensor([5])],
 [tensor([0, 1, 4]), tensor([5])],
 [tensor([0, 1, 5]), tensor([9])],
 [tensor([0, 1, 6]), tensor([9])],
 [tensor([0, 1, 7]), tensor([9])],
 [tensor([0, 1, 8]), tensor([9])],
 [tensor([0, 1, 9]), tensor([15])],
 [tensor([ 0,  1, 10]), tensor([15])],
 [tensor([ 0,  1, 11]), tensor([15])],
 [tensor([ 0,  1, 12]), tensor([15])],
 [tensor([ 0,  1, 13]), tensor([15])],
 [tensor([ 0,  1, 14]), tensor([15])],
 [tensor([ 0,  1, 15]), tensor([21])],
 [tensor([ 0,  1, 16]), tensor([21])],
 [tensor([ 0,  1, 17]), tensor([21])],
 [tensor([ 0,  1, 18]), tensor([21])],
 [tensor([ 0,  1, 19]), tensor([21])],
 [tensor([ 0,  1, 20]), tensor([21])],
 [tensor([ 0,  1, 21]), tensor([27])],
 [tensor([ 0,  1, 22]), tensor([27])],
 [tensor([ 0,  1, 23]), tensor([27])],
 [tensor([ 0,  1, 24]), tensor([27])],
 [tensor([ 0,  1, 25]), tensor([27])],
 [tensor([ 0,  1, 26]), tensor([27])],
 [tensor([ 0,  1, 27]), tensor([27])],
 [tensor([0, 2, 3

Embeddings

In [14]:
class Encoder(torch.nn.Module):
    def __init__(self, n, dm, h):
        super().__init__()
        self.h = h                  # num of attention heads
        self.dm = dm                # model dim
        self.dk = dm//h
        self.dv = dm//h                 # model dim over num of attention heads
        
        self.embedding_layer = torch.nn.Embedding(n, dm)  # n x dm
        self.norm1 = torch.nn.LayerNorm(dm)
        
        # attention
        self.wQ = torch.nn.Linear(dm, self.dk)  # dm x dk
        self.wK = torch.nn.Linear(dm, self.dk)  # dm x dk
        self.wV = torch.nn.Linear(dm, self.dv)  # dm x dv
        
        # feed forward
        self.relu = torch.nn.ReLU()
        self.ff1 = torch.nn.Linear(self.dv, dm*4)
        self.ff2 = torch.nn.Linear(dm*4, dm)
        self.norm2 = torch.nn.LayerNorm(dm)
        
        self.fc = torch.nn.Linear(n_seq*dm, len(vowels))
         
    def embedding(self, x):
        return self.embedding_layer(x) #(b x t x n) x (n x dm) -> (b x t x dm)
       
    def attention(self, x_seq):
        # TODO: Add positional embedding ?
        # Word embedding
        pos_emb = self.embedding(x_seq)
        
        # Search
        Q = self.wQ(pos_emb)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Reference
        K = self.wK(pos_emb)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Attention features
        V = self.wV(pos_emb)  # (b x t x dm) x (dm x dv) -> (b x t x dv)
        
        # Similarity
        QK = torch.matmul(Q, K.T)  # (t x dk) x (dk x t) -> (t x t) *removing batch info
        QKn = QK / torch.sqrt(torch.tensor([self.dm]))
        
        # Highlight similar words
        P = torch.nn.Softmax(dim=0)(QKn)    # (t x t)
        
        # Focus on specific features
        H = torch.matmul(P, V)              # (t x t) x (t x dv) -> (t x dv)
        
        Hn = self.norm1(H)
        
        # Attention latent space
        R = self.relu(self.ff1(Hn))  # (t x dv) x (dv x dm*4) -> (t x dm*4)
        A = self.ff2(R)  # (t x dm*4) x (dm*4 x dm) -> (t x dm)
        
        An = self.norm2(A)
        
        # Probabilities
        flatten_a = An.view(-1)                  # 1 x (t x dm)
        logits = self.fc(flatten_a)             # (t x dm) x num of labels
        p = torch.nn.Softmax(dim=0)(logits)     # prob of each label
        
        return p
    

Build

In [15]:
import random

class transformer(torch.nn.Module):
    def __init__(self, n, dm, h):
        super().__init__()
        self.encoder = Encoder(n=n, dm=dm, h=h)
        
    def forward(self, x):
        y = self.encoder.attention(x)
        
        return y

In [16]:
def one_hot_encode(t):
    one_hot_encode = torch.zeros(len(vowels))
    for i, v in enumerate(vowels):
        if v == t:
            one_hot_encode[i] = 1
    
    return one_hot_encode.float().requires_grad_(True)

Train

In [17]:
# Get number of samples
n_samples = len(samples)

# Build model
model = transformer(n=n, dm=32, h=1)

print(model)

# Define optmization algorithm
opt = torch.optim.SGD(model.parameters(), lr=0.0025, momentum=0.9)

# Define loss
cel = torch.nn.CrossEntropyLoss()

epochs = 100

for epoch in range(epochs):
    rl = 0.0
    for x_seq, t in samples:
        # avoid cumulative gradient
        opt.zero_grad()
        
        # predictions
        p = model(x_seq)

        # compute error
        vowel = characters[t.item()]
        t_enc = one_hot_encode(vowel)
        loss = cel(p, t_enc)
        
        # compute gradient
        loss.backward()
        
        # Clip the gradients
        max_grad_norm = 1.0  # Set your desired maximum gradient norm here
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # update weights
        opt.step()
        
        # running loss
        rl += loss.item()
    
    # Handcraft learnnig rate decay
    # if epoch % (0.2*epochs) == 0 and epoch > 0:
    #    for param_group in opt.param_groups:
    #        print(f"Changing learning rate to {param_group['lr']/2}")
    #        param_group['lr'] = param_group['lr']/2
    
    # avg running loss
    rl /= n_samples
    
    print(f"Epoch: {epoch+1} \tTraining Loss: {rl:.4f}")
    
print("Finished training")

transformer(
  (encoder): Encoder(
    (embedding_layer): Embedding(28, 32)
    (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (wQ): Linear(in_features=32, out_features=32, bias=True)
    (wK): Linear(in_features=32, out_features=32, bias=True)
    (wV): Linear(in_features=32, out_features=32, bias=True)
    (relu): ReLU()
    (ff1): Linear(in_features=32, out_features=128, bias=True)
    (ff2): Linear(in_features=128, out_features=32, bias=True)
    (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (fc): Linear(in_features=96, out_features=6, bias=True)
  )
)
Epoch: 1 	Training Loss: 1.2123
Epoch: 2 	Training Loss: 1.2173
Epoch: 3 	Training Loss: 1.1991
Epoch: 4 	Training Loss: 1.1888
Epoch: 5 	Training Loss: 1.1742
Epoch: 6 	Training Loss: 1.1512
Epoch: 7 	Training Loss: 1.1202
Epoch: 8 	Training Loss: 1.1085
Epoch: 9 	Training Loss: 1.1009
Epoch: 10 	Training Loss: 1.0854
Epoch: 11 	Training Loss: 1.0584
Epoch: 12 	Training Loss: 1.0484
Epoch: 13 	

In [28]:
def eval(model, samples):
    c = 0
    for sample in samples:
        p = model(sample[0])
        i = max(enumerate(p), key=lambda x: x[1])[0]
        
        predition = i2v[i]
        target = i2c[sample[1].item()]
        
        if predition == target:
            c += 1    
        
        print(predition, target)
    print(f"Acertos %:{c/len(samples)*100}")

In [19]:
eval(model, samples)

e e
e e
e e
i i
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
e e
e e
i i
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
e e
i i
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i i
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
u u
u u
u u


It seems the model is copying the training data behavior.

"Attention layer network" seems to grasp the understanding of the task.

In line four for the input 'ouz' it seems that the network is predicting a letter that has the position equal to one in the input, the letter 'u'. Maybe this is happening because there is no guarantee that the output  should not have a position before or equal the sequence. Although the dataset preserves the order, the model can finf patterns that can break sequence information. 

The decoder layer can solve this problem creating a previous information dependency through autoregression feeding. In this case vowels that came after the sequence should not be considered, since this information is referent to the past, creating a positional order constraint to next vowel generation.

For example for a current sequence 'ouz' it will never be possible to have in a previous sequence the vowels 'o, u or #', maybe this restriction can avoid the causality violation of having future information before prediction. It is like play in loto knowing for example some numbers in the sequence before it happens.

Lets create the decoder layer and see if this hypothesis make any sence.

In [20]:
class Encoder(torch.nn.Module):
    def __init__(self, n, dm, h):
        super().__init__()
        self.h = h                  # num of attention heads
        self.dm = dm                # model dim
        self.dk = dm//h
        self.dv = dm                 # model dim over num of attention heads
        
        self.embedding_layer = torch.nn.Embedding(n, dm)  # n x dm
         
        self.wQ = torch.nn.Linear(dm, self.dk)  # dm x dk
        self.wK = torch.nn.Linear(dm, self.dk)  # dm x dk
        self.wV = torch.nn.Linear(dm, self.dv)  # dm x dv
        self.norm1 = torch.nn.LayerNorm(dm)
        
        self.relu = torch.nn.ReLU()
        self.ff1 = torch.nn.Linear(self.dv, dm*4)
        self.ff2 = torch.nn.Linear(dm*4, dm)
        self.norm2 = torch.nn.LayerNorm(dm)
        
    def embedding(self, x):
        return self.embedding_layer(x) #(b x t x n) x (n x dm) -> (b x t x dm)
       
    def attention(self, x_seq):
        # Word embedding
        pos_emb = self.embedding(x_seq)
        
        # Search
        Q = self.wQ(pos_emb)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Reference
        K = self.wK(pos_emb)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Attention features
        V = self.wV(pos_emb)  # (b x t x dm) x (dm x dv) -> (b x t x dv)
        
        # Similarity
        QK = torch.matmul(Q, K.T)  # (t x dk) x (dk x t) -> (t x t) *removing batch info
        QKn = QK / torch.sqrt(torch.tensor([self.dm]))
        
        # Highlight similar words
        P = torch.nn.Softmax(dim=0)(QKn)    # (t x t)
        
        # Focus on specific features
        H = torch.matmul(P, V)              # (t x t) x (t x dv) -> (t x dv)
        Hn = self.norm1(H)
        
        # Attention latent space
        R = self.relu(self.ff1(Hn))  # (t x dv) x (dv x dm*4) -> (t x dm*4)
        A = self.ff2(R)  # (t x dm*4) x (dm*4 x dm) -> (t x dm)
        An = self.norm2(A)
        
        return An
    

In [21]:
class Decoder(torch.nn.Module):
    def __init__(self, n, dm, h):
        super().__init__()
        self._y = random.randint(0, 5)
        self.h = h                  # num of attention heads
        self.dm = dm                # model dim
        self.dk = dm//h
        self.dv = dm                 # model dim over num of attention heads
        
        self.embedding_layer = torch.nn.Embedding(n, dm)  # n x dm
        self.norm1 = torch.nn.LayerNorm(dm)
        
        self.wQ = torch.nn.Linear(dm, self.dk)  # dm x dk
        self.wK = torch.nn.Linear(dm, self.dk)  # dm x dk
        self.wV = torch.nn.Linear(dm, self.dv)  # dm x dv
        self.norm2 = torch.nn.LayerNorm(dm)
        
        self.relu = torch.nn.ReLU()
        self.ff1 = torch.nn.Linear(self.dv, dm*4)
        self.ff2 = torch.nn.Linear(dm*4, dm)
        self.norm3 = torch.nn.LayerNorm(dm)
        self.fc = torch.nn.Linear(n_seq*dm, len(vowels))
        
    def embedding(self, x):
        return self.embedding_layer(x) #(b x t x n) x (n x dm) -> (b x t x dm)
    
    def attention(self, ae, ad):
        # Search
        Q = self.wQ(ad)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Reference
        K = self.wK(ae)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Attention features
        V = self.wV(ae)  # (b x t x dm) x (dm x dv) -> (b x t x dv)
        
        # Similarity
        QK = torch.matmul(Q, K.T)  # (t x dk) x (dk x t) -> (t x t) *removing batch info
        QKn = QK / torch.sqrt(torch.tensor([self.dm]))
        
        # Highlight similar words
        P = torch.nn.Softmax(dim=0)(QKn)    # (t x t)
        
        # Focus on specific features
        H = torch.matmul(P, V)              # (t x t) x (t x dv) -> (t x dv)
        Hn = self.norm2(H)
        
        # Attention latent space
        R = self.relu(self.ff1(Hn))  # (t x dv) x (dv x dm*4) -> (t x dm*4)
        A = self.ff2(R)  # (t x dm*4) x (dm*4 x dm) -> (t x dm)
        An = self.norm3(A)
        
        # Probabilities
        flatten_a = An.view(-1)                  # 1 x (t x dm)
        logits = self.fc(flatten_a)             # (t x dm) x num of labels
        p = torch.nn.Softmax(dim=0)(logits)     # prob of each label
        
        # Get token id of output vowel
        vowel_idx = max(enumerate(p), key=lambda t: t[1])[0]
        vowel = vowels[vowel_idx]
        self._y = pos[vowel]
        
        return p
    
    def masked_attention(self, x_seq):
        # shift output embedding
        x_seq = x_seq.tolist()
        x_seq.pop(-1)
        x_seq.insert(0, self._y)
        _x_seq = torch.tensor(x_seq)
        
        # Word embedding
        pos_emb = self.embedding(_x_seq)
        
        # Search
        Q = self.wQ(pos_emb)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Reference
        K = self.wK(pos_emb)  # (b x t x dm) x (dm x dk) -> (b x t x dk)
        # Attention features
        V = self.wV(pos_emb)  # (b x t x dm) x (dm x dv) -> (b x t x dv)
        
        # Similarity
        QK = torch.matmul(Q, K.T)  # (t x dk) x (dk x t) -> (t x t) *removing batch info
        
        # Mask
        if x_seq not in valid_seqs:
            mask = torch.tensor([[0, 0, 0],
                                 [1, 1, 1],
                                 [1, 1, 1],])
        
            QK.masked_fill_(mask == 0, -1e9)
        
        QKn = QK / torch.sqrt(torch.tensor([self.dm]))
        
        # Highlight similar words
        P = torch.nn.Softmax(dim=0)(QKn)    # (t x t)
        
        # Focus on specific features
        H = torch.matmul(P, V)              # (t x t) x (t x dv) -> (t x dv)
        
        # Attention latent space
        R = self.relu(self.ff1(H))  # (t x dv) x (dv x dm*4) -> (t x dm*4)
        A = self.ff2(R)  # (t x dm*4) x (dm*4 x dm) -> (t x dm)
        An = self.norm1(H)
        
        return An

In [22]:
class transformer(torch.nn.Module):
    def __init__(self, n, dm, h):
        super().__init__()
        self.encoder = Encoder(n=n, dm=dm, h=h)
        self.decoder = Decoder(n=n, dm=dm, h=h)
        
    def forward(self, x):
        # encoder
        ae = self.encoder.attention(x)
        # decoder
        ad = self.decoder.masked_attention(x)
        y = self.decoder.attention(ae, ad)
            
        return y

In [23]:
# Get number of samples
n_samples = len(samples)

# Build model
model = transformer(n=n, dm=16, h=1)

print(model)

# Define optmization algorithm
opt = torch.optim.SGD(model.parameters(), lr=0.0025, momentum=0.9)

# Define loss
cel = torch.nn.CrossEntropyLoss()

epochs = 100

for epoch in range(epochs):
    rl = 0.0
    for x_seq, t in samples:
        # avoid cumulative gradient
        opt.zero_grad()
        
        # predictions
        p = model(x_seq)

        # compute error
        vowel = characters[t.item()]
        t_enc = one_hot_encode(vowel)
        loss = cel(p, t_enc)
        
        # compute gradient
        loss.backward()
        
        # Clip the gradients
        max_grad_norm = 1.0  # Set your desired maximum gradient norm here
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)


        # update weights
        opt.step()
        
        # running loss
        rl += loss.item()
    
    # Handcraft learnnig rate decay
    # if epoch % (0.2*epochs) == 0 and epoch > 0:
    #    for param_group in opt.param_groups:
    #        print(f"Changing learning rate to {param_group['lr']/2}")
    #        param_group['lr'] = param_group['lr']/2
    
    # avg running loss
    rl /= n_samples
    
    print(f"Epoch: {epoch+1} \tTraining Loss: {rl:.4f}")
    
print("Finished training")

transformer(
  (encoder): Encoder(
    (embedding_layer): Embedding(28, 16)
    (wQ): Linear(in_features=16, out_features=16, bias=True)
    (wK): Linear(in_features=16, out_features=16, bias=True)
    (wV): Linear(in_features=16, out_features=16, bias=True)
    (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (relu): ReLU()
    (ff1): Linear(in_features=16, out_features=64, bias=True)
    (ff2): Linear(in_features=64, out_features=16, bias=True)
    (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): Decoder(
    (embedding_layer): Embedding(28, 16)
    (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (wQ): Linear(in_features=16, out_features=16, bias=True)
    (wK): Linear(in_features=16, out_features=16, bias=True)
    (wV): Linear(in_features=16, out_features=16, bias=True)
    (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (relu): ReLU()
    (ff1): Linear(in_features=16, out_features=64, bias=True)
   

Epoch: 1 	Training Loss: 1.3792
Epoch: 2 	Training Loss: 1.2380
Epoch: 3 	Training Loss: 1.2272
Epoch: 4 	Training Loss: 1.2161
Epoch: 5 	Training Loss: 1.2074
Epoch: 6 	Training Loss: 1.1981
Epoch: 7 	Training Loss: 1.1685
Epoch: 8 	Training Loss: 1.1295
Epoch: 9 	Training Loss: 1.1090
Epoch: 10 	Training Loss: 1.1016
Epoch: 11 	Training Loss: 1.0953
Epoch: 12 	Training Loss: 1.0863
Epoch: 13 	Training Loss: 1.0774
Epoch: 14 	Training Loss: 1.0738
Epoch: 15 	Training Loss: 1.0727
Epoch: 16 	Training Loss: 1.0716
Epoch: 17 	Training Loss: 1.0717
Epoch: 18 	Training Loss: 1.0710
Epoch: 19 	Training Loss: 1.0717
Epoch: 20 	Training Loss: 1.0700
Epoch: 21 	Training Loss: 1.0708
Epoch: 22 	Training Loss: 1.0704
Epoch: 23 	Training Loss: 1.0692
Epoch: 24 	Training Loss: 1.0688
Epoch: 25 	Training Loss: 1.0677
Epoch: 26 	Training Loss: 1.0653
Epoch: 27 	Training Loss: 1.0618
Epoch: 28 	Training Loss: 1.0564
Epoch: 29 	Training Loss: 1.0550
Epoch: 30 	Training Loss: 1.0550
Epoch: 31 	Training

In [29]:
# target x
eval(model, samples)

o e
o e
o e
o i
o i
o i
o i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i e
i e
i i
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i e
i i
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i i
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i i
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i i
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
i i
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
o o
u u
u u
u u
u u
u u
u u
# #
# #
# #
# #
# #
# #
# #
o o
u u
u u
u u


In [32]:
characters = list(reversed("-abcdefghijklmnopqrstuvwxyz#"))


'<reversed object at 0x7f7248669f10>'

In [33]:
characters = list(reversed("-abcdefghijklmnopqrstuvwxyz#"))
emb = {}
pos = {}
i2c = {}
for i, c in enumerate(characters):
    pos[c] = i         # position i mapping
    i2c[i] = c
    #emb[c] = format(i,'05b') # binary encoding mapping
    
def generate_combinations(characters, length):
    def generate_combinations_recursive(current_combination, remaining_characters):
        if len(current_combination) == length:
            combinations.append(current_combination)
            return
        for i in range(len(remaining_characters)):
            generate_combinations_recursive(
                current_combination + remaining_characters[i],
                remaining_characters[i + 1:]
            )

    combinations = []
    generate_combinations_recursive('', characters)
    return combinations

length = 3

combinations = generate_combinations(characters, length)

for combo in combinations:
    print(combo)
  

#zy
#zx
#zw
#zv
#zu
#zt
#zs
#zr
#zq
#zp
#zo
#zn
#zm
#zl
#zk
#zj
#zi
#zh
#zg
#zf
#ze
#zd
#zc
#zb
#za
#z-
#yx
#yw
#yv
#yu
#yt
#ys
#yr
#yq
#yp
#yo
#yn
#ym
#yl
#yk
#yj
#yi
#yh
#yg
#yf
#ye
#yd
#yc
#yb
#ya
#y-
#xw
#xv
#xu
#xt
#xs
#xr
#xq
#xp
#xo
#xn
#xm
#xl
#xk
#xj
#xi
#xh
#xg
#xf
#xe
#xd
#xc
#xb
#xa
#x-
#wv
#wu
#wt
#ws
#wr
#wq
#wp
#wo
#wn
#wm
#wl
#wk
#wj
#wi
#wh
#wg
#wf
#we
#wd
#wc
#wb
#wa
#w-
#vu
#vt
#vs
#vr
#vq
#vp
#vo
#vn
#vm
#vl
#vk
#vj
#vi
#vh
#vg
#vf
#ve
#vd
#vc
#vb
#va
#v-
#ut
#us
#ur
#uq
#up
#uo
#un
#um
#ul
#uk
#uj
#ui
#uh
#ug
#uf
#ue
#ud
#uc
#ub
#ua
#u-
#ts
#tr
#tq
#tp
#to
#tn
#tm
#tl
#tk
#tj
#ti
#th
#tg
#tf
#te
#td
#tc
#tb
#ta
#t-
#sr
#sq
#sp
#so
#sn
#sm
#sl
#sk
#sj
#si
#sh
#sg
#sf
#se
#sd
#sc
#sb
#sa
#s-
#rq
#rp
#ro
#rn
#rm
#rl
#rk
#rj
#ri
#rh
#rg
#rf
#re
#rd
#rc
#rb
#ra
#r-
#qp
#qo
#qn
#qm
#ql
#qk
#qj
#qi
#qh
#qg
#qf
#qe
#qd
#qc
#qb
#qa
#q-
#po
#pn
#pm
#pl
#pk
#pj
#pi
#ph
#pg
#pf
#pe
#pd
#pc
#pb
#pa
#p-
#on
#om
#ol
#ok
#oj
#oi
#oh
#og
#of
#oe
#od
#oc
#ob
#oa
#o-
#nm
#nl
#nk
#nj


In [ ]:
for sample in combinations:
    p = model(sample)
    i = max(enumerate(p), key=lambda x: x[1])[0]
    
    predition = i2v[i]
    target = i2c[sample[1].item()]
    
    if predition == target:
        c += 1    